- cancel lstm
- add xgb

In [1]:
!pip install --no-index --find-links /kaggle/input/pytorchtabnet/pytorch_tabnet-2.0.0-py3-none-any.whl pytorch-tabnet

Looking in links: /kaggle/input/pytorchtabnet/pytorch_tabnet-2.0.0-py3-none-any.whl
Processing /kaggle/input/pytorchtabnet/pytorch_tabnet-2.0.0-py3-none-any.whl


In [2]:
import sys
!cp ../input/rapids/rapids.0.15.0 /opt/conda/envs/rapids.tar.gz
!cd /opt/conda/envs/ && tar -xzvf rapids.tar.gz > /dev/null
sys.path = ["/opt/conda/envs/rapids/lib/python3.7/site-packages"] + sys.path
sys.path = ["/opt/conda/envs/rapids/lib/python3.7"] + sys.path
sys.path = ["/opt/conda/envs/rapids/lib"] + sys.path
!cp /opt/conda/envs/rapids/lib/libxgboost.so /opt/conda/lib/

In [3]:
import os
import warnings
import numpy as np
import pandas as pd 
from xgboost import XGBClassifier
from sklearn import preprocessing
from sklearn.metrics import log_loss
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from tqdm import tqdm_notebook as tqdm
from category_encoders import CountEncoder
from sklearn.feature_selection import VarianceThreshold
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import KFold, StratifiedKFold
sys.path.append('../input/multilabelstraifier/')
from ml_stratifiers import MultilabelStratifiedKFold
warnings.filterwarnings('ignore')
import xgboost as xgb

import time
import torch
import random
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import tensorflow as tf
from torch.nn.modules.loss import _WeightedLoss

from sklearn.linear_model import LogisticRegression
from cuml.svm import SVC, SVR

from pytorch_tabnet.metrics import Metric
from pytorch_tabnet.tab_model import TabNetRegressor

# Preprocess

In [4]:
DATA_DIR = '/kaggle/input/lish-moa/'
train = pd.read_csv(DATA_DIR + 'train_features.csv')
targets = pd.read_csv(DATA_DIR + 'train_targets_scored.csv')
test = pd.read_csv(DATA_DIR + 'test_features.csv')

In [5]:
target_feats = [ i for i in targets.columns if i != "sig_id"]
g_feats = [i for i in train.columns if "g-" in i]
c_feats = [i for i in train.columns if "c-" in i]

In [6]:
cons_train_index = train[train.cp_type!="ctl_vehicle"].index
noncons_test_index = test[test.cp_type=="ctl_vehicle"].index
cons_test_index = test[test.cp_type!="ctl_vehicle"].index

test = test[test.index.isin(cons_test_index)].reset_index(drop=True)
train = train[train.index.isin(cons_train_index)].reset_index(drop=True)
fn_targets = targets.drop("sig_id", axis=1).copy()
fn_targets = fn_targets[fn_targets.index.isin(cons_train_index)].copy().reset_index(drop=True).to_numpy()
y = targets.drop("sig_id", axis=1).copy()

# Feature engineering

In [7]:
def fe_simple(df, remove_features):
    tmp = df.copy()
    tmp.loc[:, 'cp_dose'] = tmp.loc[:, 'cp_dose'].map({'D1': 0, 'D2': 1})
    tmp.drop(remove_features, axis=1, inplace=True)
    return tmp

def fe_xgb(df):
    tmp = df.copy()
    tmp['g_sum'] = tmp[g_feats].sum(axis = 1)
    tmp['g_mean'] = tmp[g_feats].mean(axis = 1)
    tmp['g_std'] = tmp[g_feats].std(axis = 1)
    tmp['g_kurt'] = tmp[g_feats].kurtosis(axis = 1)
    tmp['g_skew'] = tmp[g_feats].skew(axis = 1)
    tmp['c_sum'] = tmp[c_feats].sum(axis = 1)
    tmp['c_mean'] = tmp[c_feats].mean(axis = 1)
    tmp['c_std'] = tmp[c_feats].std(axis = 1)
    tmp['c_kurt'] = tmp[c_feats].kurtosis(axis = 1)
    tmp['c_skew'] = tmp[c_feats].skew(axis = 1)
    tmp['gc_sum'] = tmp[c_feats + g_feats].sum(axis = 1)
    tmp['gc_mean'] = tmp[c_feats + g_feats].mean(axis = 1)
    tmp['gc_std'] = tmp[c_feats + g_feats].std(axis = 1)
    tmp['gc_kurt'] = tmp[c_feats + g_feats].kurtosis(axis = 1)
    tmp['gc_skew'] = tmp[c_feats + g_feats].skew(axis = 1)
    return tmp

def fe_mlp(df_train, df_test):
    tmp_train = df_train.copy()
    tmp_test = df_test.copy()
    X = tmp_train.iloc[:,4:].copy().values
    select = VarianceThreshold(threshold=0.7)
    X_new = select.fit_transform(X)
    drop_feats = list(np.array(tmp_train.iloc[:,4:].columns)[select.get_support()==False])
    
    tmp_train.drop(drop_feats, axis=1, inplace=True)
    tmp_test.drop(drop_feats, axis=1, inplace=True)

    modg_feats = [i for i in tmp_train.columns if "g-" in i]
    modc_feats = [i for i in tmp_train.columns if "c-" in i]
    
    for i in modc_feats + modg_feats:
        ss = preprocessing.QuantileTransformer(n_quantiles=1000, random_state=0, output_distribution="normal")
        ss.fit(tmp_train[i].values.reshape(-1,1))
        tmp_train[i] = ss.transform(tmp_train[i].values.reshape(-1,1))
        tmp_test[i] = ss.transform(tmp_test[i].values.reshape(-1,1))
    
    c_num = 10
    pca_c_cols = ["pca-c"+str(i+1) for i in range(c_num)]
    pca = PCA(n_components=c_num,random_state=42)
    c_train = pca.fit_transform(tmp_train[modc_feats])
    c_test = pca.transform(tmp_test[modc_feats])
    c_train = pd.DataFrame(c_train, columns=pca_c_cols)
    c_test = pd.DataFrame(c_test, columns=pca_c_cols)

    g_num = 60
    pca_g_cols = ["pca-g"+str(i+1) for i in range(g_num)]
    pca = PCA(n_components=g_num, random_state=42)
    g_train = pca.fit_transform(tmp_train[modg_feats])
    g_test = pca.transform(tmp_test[modg_feats])
    g_train = pd.DataFrame(g_train, columns=pca_g_cols)
    g_test = pd.DataFrame(g_test, columns=pca_g_cols)

    tmp_train = pd.concat([tmp_train, c_train],axis=1)
    tmp_test = pd.concat([tmp_test, c_test],axis=1)
    tmp_train = pd.concat([tmp_train, g_train],axis=1)
    tmp_test = pd.concat([tmp_test, g_test],axis=1)
    
    return tmp_train, tmp_test

def fe_mlp2(df):
    tmp = df.copy()
    modg_feats = [i for i in tmp.columns if "g-" in i]
    modc_feats = [i for i in tmp.columns if "c-" in i]
    tmp['g_sum'] = tmp[modg_feats].sum(axis = 1)
    tmp['g_mean'] = tmp[modg_feats].mean(axis = 1)
    tmp['g_std'] = tmp[modg_feats].std(axis = 1)
    tmp['g_kurt'] = tmp[modg_feats].kurtosis(axis = 1)
    tmp['g_skew'] = tmp[modg_feats].skew(axis = 1)
    tmp['c_sum'] = tmp[modc_feats].sum(axis = 1)
    tmp['c_mean'] = tmp[modc_feats].mean(axis = 1)
    tmp['c_std'] = tmp[modc_feats].std(axis = 1)
    tmp['c_kurt'] = tmp[modc_feats].kurtosis(axis = 1)
    tmp['c_skew'] = tmp[modc_feats].skew(axis = 1)
    tmp['gc_sum'] = tmp[modc_feats + modg_feats].sum(axis = 1)
    tmp['gc_mean'] = tmp[modc_feats + modg_feats].mean(axis = 1)
    tmp['gc_std'] = tmp[modc_feats + modg_feats].std(axis = 1)
    tmp['gc_kurt'] = tmp[modc_feats + modg_feats].kurtosis(axis = 1)
    tmp['gc_skew'] = tmp[modc_feats + modg_feats].skew(axis = 1)
    tmp = pd.get_dummies(tmp, columns=['cp_time','cp_dose'])
    tmp.drop(["cp_type", "sig_id"], axis=1, inplace=True) 
    return tmp

def fe_tabnet(df_train, df_test):
    tmp_train = df_train.copy()
    tmp_test = df_test.copy()
    X = tmp_train.iloc[:,4:].copy().values
    select = VarianceThreshold(threshold=0.7)
    X_new = select.fit_transform(X)
    drop_feats = list(np.array(tmp_train.iloc[:,4:].columns)[select.get_support()==False])
    
    tmp_train.drop(drop_feats, axis=1, inplace=True)
    tmp_test.drop(drop_feats, axis=1, inplace=True)

    modg_feats = [i for i in tmp_train.columns if "g-" in i]
    modc_feats = [i for i in tmp_train.columns if "c-" in i]
    
    for i in modc_feats + modg_feats:
        ss = preprocessing.QuantileTransformer(n_quantiles=1000, random_state=0, output_distribution="normal")
        ss.fit(tmp_train[i].values.reshape(-1,1))
        tmp_train[i] = ss.transform(tmp_train[i].values.reshape(-1,1))
        tmp_test[i] = ss.transform(tmp_test[i].values.reshape(-1,1))
        
    tmp_train = pd.get_dummies(tmp_train, columns=['cp_time','cp_dose'])
    tmp_train.drop(["cp_type", "sig_id"], axis=1, inplace=True)
    tmp_test = pd.get_dummies(tmp_test, columns=['cp_time','cp_dose'])
    tmp_test.drop(["cp_type", "sig_id"], axis=1, inplace=True)
    return tmp_train, tmp_test

def fe_lstm(df):
    tmp = df.copy()    
    tmp.drop(["cp_type", "sig_id", "cp_dose", "cp_time"], axis=1, inplace=True)
    return tmp

remove_features = ["cp_type" , "sig_id"]

In [8]:
fn_train = fe_simple(train, remove_features)
fn_test = fe_simple(test, remove_features)

# xgb --------------------------
X = fe_new(fn_train)
X_test = fe_new(fn_test)

# pytorch mlp -----------------------------------
mlp_train, mlp_test = fe_mlp(train, test)
mlp_train = fe_mlp2(mlp_train).to_numpy()
mlp_test = fe_mlp2(mlp_test).to_numpy()

# pytorch tabnet ----------------------------------
py_train = fn_train.copy()
py_test = fn_test.copy()

ss = preprocessing.RobustScaler()
py_train= ss.fit_transform(py_train)
py_test = ss.transform(py_test)

# pytorch lstm -------------------------------
lstm_train = fe_lstm(train)
lstm_test = fe_lstm(test)
ss = preprocessing.RobustScaler()
lstm_train= ss.fit_transform(lstm_train)
lstm_test = ss.transform(lstm_test)

# svm-----------------------
ss = preprocessing.StandardScaler()
fn_train= ss.fit_transform(fn_train)
fn_test = ss.transform(fn_test)

NameError: name 'fe_new' is not defined

# 1st xgb

In [9]:
np.random.seed(42)
NFOLDS=5

classifier = MultiOutputClassifier(XGBClassifier(tree_method='gpu_hist'))

clf = Pipeline([('classify', classifier)
               ])

params = {'classify__estimator__gamma': 3.6975,
          'classify__estimator__learning_rate': 0.0803,
          'classify__estimator__max_delta_step': 2.0706,
          'classify__estimator__max_depth': 10,
          'classify__estimator__min_child_weight': 31.5800,
          'classify__estimator__n_estimators': 166,
         }

clf.set_params(**params)

Pipeline(steps=[('classify',
                 MultiOutputClassifier(estimator=XGBClassifier(base_score=None,
                                                               booster=None,
                                                               colsample_bylevel=None,
                                                               colsample_bynode=None,
                                                               colsample_bytree=None,
                                                               gamma=3.6975,
                                                               gpu_id=None,
                                                               importance_type='gain',
                                                               interaction_constraints=None,
                                                               learning_rate=0.0803,
                                                               max_delta_step=2.0706,
                                                   

In [10]:
xgb1_oof = np.zeros((X.shape[0], y.shape[1]))
xgb1_test = np.zeros((X_test.shape[0], y.shape[1]))
mskf = MultilabelStratifiedKFold(n_splits=NFOLDS, random_state=42, shuffle=True)
for fn, (trn_idx, val_idx) in enumerate(mskf.split(X, fn_targets)):
    X_train, X_val = X.iloc[trn_idx,:].to_numpy(), X.iloc[val_idx,:].to_numpy()
    y_train, y_val = fn_targets[trn_idx], fn_targets[val_idx]
    
    clf.fit(X_train, y_train)
        
    val_preds = clf.predict_proba(X_val) # list of preds per class
    val_preds = np.array(val_preds)[:,:,1].T # take the positive class
    xgb1_oof[val_idx] = val_preds
    loss = log_loss(np.ravel(y_val), np.ravel(val_preds))
    print(loss)
        
    preds = clf.predict_proba(X_test)
    preds = np.array(preds)[:,:,1].T # take the positive class
    xgb1_test += preds / NFOLDS

NameError: name 'X' is not defined

In [11]:
check_xgb = np.zeros([targets.shape[0], targets.shape[1]-1])
check_xgb[cons_train_index,:] =  xgb1_oof
print('OOF log loss: ', log_loss(np.ravel(y), np.ravel(check_xgb)))

NameError: name 'xgb1_oof' is not defined

# 1st pytorch

In [12]:
class SmoothCrossEntropyLoss(_WeightedLoss):
    def __init__(self, weight=None, reduction='mean', smoothing=0.0):
        super().__init__(weight=weight, reduction=reduction)
        self.smoothing = smoothing
        self.weight = weight
        self.reduction = reduction

    @staticmethod
    def _smooth(targets, n_classes, smoothing=0.0):
        assert 0 <= smoothing <= 1
        with torch.no_grad():
            targets = targets * (1 - smoothing) + torch.ones_like(targets).to(device) * smoothing / n_classes
        return targets

    def forward(self, inputs, targets):
        targets = SmoothCrossEntropyLoss()._smooth(targets, inputs.shape[1], self.smoothing)

        if self.weight is not None:
            inputs = inputs * self.weight.unsqueeze(0)

        loss = F.binary_cross_entropy_with_logits(inputs, targets)

        return loss

In [13]:
device = "cuda" if torch.cuda.is_available() else "cpu"
batch_size = 128
n_folds=7
train_epochs = 20
smoothing = 0.001
p_min = smoothing
p_max = 1 - smoothing

def mean_log_loss(y_true, y_pred):
    metrics = []
    for i, target in enumerate(target_feats):
        metrics.append(log_loss(y_true[:, i], y_pred[:, i].astype(float), labels=[0,1]))
    return np.mean(metrics)

def seed_everything(seed=1234): 
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    tf.random.set_seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
class MoaModel(nn.Module):
    def __init__(self, num_columns, last_num):
        super(MoaModel, self).__init__()
        self.batch_norm1 = nn.BatchNorm1d(num_columns)
        self.dropout1 = nn.Dropout(0.1)
        self.dense1 = nn.utils.weight_norm(nn.Linear(num_columns, 1024))
        self.relu1 = nn.LeakyReLU()
        
        self.batch_norm2 = nn.BatchNorm1d(1024)
        self.dropout2 = nn.Dropout(0.1)
        self.dense2 = nn.utils.weight_norm(nn.Linear(1024, 1024))
        self.relu2 = nn.LeakyReLU()
        
        self.batch_norm3 = nn.BatchNorm1d(1024)
        self.dropout3 = nn.Dropout(0.1)
        self.dense3 = nn.utils.weight_norm(nn.Linear(1024, last_num))
        
    def forward(self, x):
        x = self.batch_norm1(x)
        x = self.dropout1(x)
        x = self.relu1(self.dense1(x))
        
        x = self.batch_norm2(x)
        x = self.dropout2(x)
        x = self.relu2(self.dense2(x))
        
        x = self.batch_norm3(x)
        x = self.dropout3(x)
        x = self.dense3(x)
        
        return x
    
def modelling_torch(X_train, y_train, X_test, sample_seed, last_num):
    seed_everything(seed=sample_seed) 

    test_len = X_test.shape[0]
    
    mskf=MultilabelStratifiedKFold(n_splits = n_folds, shuffle=True, random_state=224)
    metric = lambda inputs, targets : F.binary_cross_entropy((torch.clamp(torch.sigmoid(inputs), p_min, p_max)), targets)

    models = []
    
    X_test2 = torch.tensor(X_test, dtype=torch.float32)
    test = torch.utils.data.TensorDataset(X_test2) 
    test_loader = torch.utils.data.DataLoader(test, batch_size=batch_size, shuffle=False)
    
    oof = np.zeros([len(X_train),y_train.shape[1]])
    oof_targets = np.zeros([len(X_train),y_train.shape[1]])
    pred_value = np.zeros([test_len, y_train.shape[1]])
    scores = []
    for fold, (train_index, valid_index) in enumerate(mskf.split(X_train, y_train)):
        print("Fold "+str(fold+1))
        X_train2 = torch.tensor(X_train[train_index,:], dtype=torch.float32)
        X_valid2 = torch.tensor(X_train[valid_index,:], dtype=torch.float32)

        y_train2 = torch.tensor(y_train[train_index], dtype=torch.float32)
        y_valid2 = torch.tensor(y_train[valid_index], dtype=torch.float32)
        
        train = torch.utils.data.TensorDataset(X_train2, y_train2)
        valid = torch.utils.data.TensorDataset(X_valid2, y_valid2)
        
        train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True) 
        valid_loader = torch.utils.data.DataLoader(valid, batch_size=batch_size, shuffle=False)
            
        clf = MoaModel(mlp_train.shape[1], last_num)
        loss_fn = SmoothCrossEntropyLoss(smoothing=smoothing)
        
        optimizer = optim.Adam(clf.parameters(), lr = 0.001, weight_decay=1e-5) 
        scheduler = optim.lr_scheduler.OneCycleLR(optimizer=optimizer, pct_start=0.1, div_factor=1e3, 
                                              max_lr=1e-2, epochs=train_epochs, steps_per_epoch=len(train_loader))
        
        clf.to(device)
        
        best_val_loss = np.inf
        
        for epoch in range(train_epochs):
            clf.train()
            sm_avg_loss = 0.
            for x_batch, y_batch in tqdm(train_loader, disable=True):
                x_batch = x_batch.to(device)
                y_batch = y_batch.to(device)
                y_pred = clf(x_batch) 
                loss = loss_fn(y_pred, y_batch)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                scheduler.step()
                sm_avg_loss += metric(y_pred, y_batch) / len(train_loader) 
                
            clf.eval()
            sm_avg_val_loss = 0.
            
            for i, (x_batch, y_batch) in enumerate(valid_loader): 
                x_batch = x_batch.to(device)
                y_batch = y_batch.to(device)
                y_pred = clf(x_batch).detach()
                sm_avg_val_loss += metric(y_pred, y_batch) / len(valid_loader)
                            
            if sm_avg_val_loss < best_val_loss:
                best_val_loss = sm_avg_val_loss
                print('Epoch {} sm_loss={:.5f}  sm_val_loss={:.5f}'.format(epoch + 1, sm_avg_loss, sm_avg_val_loss))
                torch.save(clf.state_dict(), 'best-model-parameters.pt')
                
        pred_model = MoaModel(mlp_train.shape[1], last_num)
        pred_model.load_state_dict(torch.load('best-model-parameters.pt'))         
        pred_model.eval()
        
        # validation check ----------------
        oof_epoch = np.zeros([X_valid2.size(0), y_train.shape[1]])
        target_epoch = np.zeros([X_valid2.size(0), y_train.shape[1]])
        for i, (x_batch, y_batch) in enumerate(valid_loader): 
            y_pred = pred_model(x_batch).detach()
            oof_epoch[i * batch_size:(i+1) * batch_size,:] = torch.clamp(torch.sigmoid(y_pred.cpu()), p_min, p_max)
            target_epoch[i * batch_size:(i+1) * batch_size,:] = y_batch.cpu().numpy()
        oof[valid_index,:] = oof_epoch
        oof_targets[valid_index,:] = target_epoch
        #-----------------------------------
        
        # test predcition --------------
        test_preds = np.zeros([test_len, y_train.shape[1]])
        for i, (x_batch,) in enumerate(test_loader): 
            y_pred = pred_model(x_batch).detach()
            test_preds[i * batch_size:(i+1) * batch_size, :] = torch.clamp(torch.sigmoid(y_pred.cpu()), p_min, p_max)
        pred_value += test_preds / n_folds
        # ------------------------------
        
    print("Seed {}".format(seed_))
    print("Total log loss: {}".format(mean_log_loss(oof_targets, oof)))
    
    return oof, pred_value

In [14]:
seeds = [0,1,2,3,4]
pytorch1_oof = np.zeros([len(mlp_train),fn_targets.shape[1]])
pytorch1_test = np.zeros([len(mlp_test),fn_targets.shape[1]])

for seed_ in seeds:
    oof, pytorch_pred = modelling_torch(mlp_train, fn_targets, mlp_test, seed_, fn_targets.shape[1])
    pytorch1_oof += oof / len(seeds)
    pytorch1_test += pytorch_pred / len(seeds)

NameError: name 'mlp_train' is not defined

In [15]:
check_mlp = np.zeros([targets.shape[0], targets.shape[1]-1])
check_mlp[cons_train_index,:] = pytorch1_oof
print('OOF log loss: ', log_loss(np.ravel(y), np.ravel(check_mlp)))

NameError: name 'pytorch1_oof' is not defined

# 1st tabnet

In [16]:
class LogitsLogLoss(Metric):
    def __init__(self):
        self._name = "logits_ll"
        self._maximize = False

    def __call__(self, y_true, y_pred):
        logits = 1 / (1 + np.exp(-y_pred))
        aux = (1-y_true)*np.log(1-logits+1e-15) + y_true*np.log(logits+1e-15)
        return np.mean(-aux)

In [17]:
MAX_EPOCH=200

def seed_tabnet_everything(seed_value):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    os.environ['PYTHONHASHSEED'] = str(seed_value)
    
    if torch.cuda.is_available(): 
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
        
def modelling_tabnet(tr, target, te, sample_seed):
    seed_tabnet_everything(sample_seed) 
    tabnet_params = dict(n_d=12, n_a=12, n_steps=1, gamma=1.3, seed = sample_seed,
                     lambda_sparse=0, optimizer_fn=torch.optim.Adam,
                     optimizer_params=dict(lr=2e-2, weight_decay=1e-5),
                     mask_type='entmax',
                     scheduler_params=dict(mode="min",
                                           patience=5,
                                           min_lr=1e-5,
                                           factor=0.9,),
                     scheduler_fn=torch.optim.lr_scheduler.ReduceLROnPlateau,
                     verbose=10,
                     )
    test_cv_preds = []
    
    NB_SPLITS = 5
    mskf = MultilabelStratifiedKFold(n_splits=NB_SPLITS, random_state=0, shuffle=True)
    oof_preds = np.zeros([len(tr),target.shape[1]])
    for fold_nb, (train_idx, val_idx) in enumerate(mskf.split(train, target)):
        print("FOLDS : ", fold_nb+1)

        ## model
        X_train, y_train = tr[train_idx, :], target[train_idx, :]
        X_val, y_val = tr[val_idx, :], target[val_idx, :]
        model = TabNetRegressor(**tabnet_params)
        
        model.fit(X_train=X_train,
              y_train=y_train,
              eval_set=[(X_val, y_val)],
              eval_name = ["val"],
              eval_metric = ["logits_ll"],
              max_epochs=MAX_EPOCH,
              patience=20, batch_size=1024, virtual_batch_size=128,
              num_workers=1, drop_last=False,
              # use binary cross entropy as this is not a regression problem
              loss_fn=torch.nn.functional.binary_cross_entropy_with_logits)
        
        preds_val = model.predict(X_val)
        preds =  1 / (1 + np.exp(-preds_val))
        oof_preds[val_idx,:] = preds
        
        # preds on test
        preds_test = model.predict(te)
        test_cv_preds.append(1 / (1 + np.exp(-preds_test)))

    test_preds_all = np.stack(test_cv_preds)
    return oof_preds, test_preds_all

In [18]:
tabnet1_oof, tabnet1_test = modelling_tabnet(py_train, fn_targets, py_test, 0)

NameError: name 'py_train' is not defined

In [19]:
tabnet1_test = tabnet1_test.mean(axis=0)
check_tabnet = np.zeros([targets.shape[0], targets.shape[1]-1])
check_tabnet[cons_train_index,:] = tabnet1_oof
print('OOF log loss: ', log_loss(np.ravel(y), np.ravel(check_tabnet)))

NameError: name 'tabnet1_test' is not defined

# 1st lstm

In [20]:
class myLSTM(nn.Module):
    def __init__(self, last_num):
        super().__init__()

        self.lstm_hidden_size = 772
        self.c_lstm_hidden_size = 100
        self.g_layer_num = 1
        self.c_layer_num = 1
        
        self.hidden_dim = 512
        self.hidden_dim_c = 10
        
        self.lstm = nn.LSTM(self.lstm_hidden_size, self.hidden_dim, batch_first=True, bidirectional=True, num_layers=self.g_layer_num)
        self.c_lstm = nn.LSTM(self.c_lstm_hidden_size, self.hidden_dim_c, batch_first=True, bidirectional=True, num_layers=self.c_layer_num)

        self.dropout = nn.Dropout(0.1)
        self.out = nn.utils.weight_norm(nn.Linear((self.hidden_dim+self.hidden_dim_c) * 2 , last_num))
        self.batch_norm = nn.BatchNorm1d((self.hidden_dim+self.hidden_dim_c) * 2)
        
    def forward(self, cont_g, cont_c): 
        cont_g = torch.unsqueeze(cont_g, 1)
        h_lstm, lstm_out = self.lstm(cont_g) # h_lstm: 256 * 1 * (2 * 512)
        conc_g = h_lstm.view(-1, self.hidden_dim * 2)
        
        cont_c = torch.unsqueeze(cont_c, 1)
        h_lstm_c, lstm_out_c = self.c_lstm(cont_c) # h_lstm: 256 * 1 * (2 * 5)
        conc_c = h_lstm_c.view(-1, self.hidden_dim_c * 2)
        
        conc = torch.cat((conc_g, conc_c),1)
        conc = self.batch_norm(conc)
        
        dropped = self.dropout(conc)
        out = self.out(dropped)
        return out

In [21]:
batch_size = 256
train_epochs = 20
n_folds=7

device = "cuda" if torch.cuda.is_available() else "cpu"

def seed_everything(seed=1234): 
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    tf.random.set_seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
def modelling_lstm(X_train, y_train, X_test, sample_seed, last_num):
    
    mskf=MultilabelStratifiedKFold(n_splits = n_folds, shuffle=True, random_state=2)
    
    seed_everything(seed=sample_seed) 
    
    test_len = X_test.shape[0]
    
    models = []
    
    X_test_g = torch.tensor(X_test[:,:772], dtype=torch.float32)
    X_test_c = torch.tensor(X_test[:,772:872], dtype=torch.float32)

    X_test = torch.utils.data.TensorDataset(X_test_g, X_test_c) 
    test_loader = torch.utils.data.DataLoader(X_test, batch_size=batch_size, shuffle=False)
    
    oof = np.zeros([len(X_train),y_train.shape[1]])
    oof_targets = np.zeros([len(X_train),y_train.shape[1]])
    pred_value = np.zeros([test_len, y_train.shape[1]])
    scores = []
    
    for fold, (train_index, valid_index) in enumerate(mskf.split(X_train, y_train)):
        print("Fold "+str(fold+1))
        X_train2_g = torch.tensor(X_train[train_index,:772], dtype=torch.float32)
        X_valid2_g = torch.tensor(X_train[valid_index,:772], dtype=torch.float32)
        X_train2_c = torch.tensor(X_train[train_index,772:872], dtype=torch.float32)
        X_valid2_c = torch.tensor(X_train[valid_index,772:872], dtype=torch.float32)
        
        y_train2 = torch.tensor(y_train[train_index], dtype=torch.float32)
        y_valid2 = torch.tensor(y_train[valid_index], dtype=torch.float32)
            
        clf = myLSTM(last_num)
        loss_fn = torch.nn.BCEWithLogitsLoss() 

        optimizer = optim.Adam(clf.parameters(), lr = 0.01, weight_decay=1e-5) 
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3, eps=1e-4, verbose=True)
        
        train = torch.utils.data.TensorDataset(X_train2_g, X_train2_c, y_train2)
        valid = torch.utils.data.TensorDataset(X_valid2_g, X_valid2_c, y_valid2)
        
        clf.to(device)
        
        train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True) 
        valid_loader = torch.utils.data.DataLoader(valid, batch_size=batch_size, shuffle=False)
        
        best_val_loss = np.inf
        for epoch in range(train_epochs):
            start_time = time.time()
            clf.train()
            avg_loss = 0.
            for x_batch_g, x_batch_c, y_batch in tqdm(train_loader, disable=True):
                x_batch_g = x_batch_g.to(device)
                x_batch_c = x_batch_c.to(device)
                y_batch = y_batch.to(device)
                y_pred = clf(x_batch_g, x_batch_c) 
                loss = loss_fn(y_pred, y_batch)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                avg_loss += loss.item() / len(train_loader) 
                
            clf.eval()
            avg_val_loss = 0.
            for i, (x_batch_g, x_batch_c, y_batch) in enumerate(valid_loader): 
                x_batch_g = x_batch_g.to(device)
                x_batch_c = x_batch_c.to(device)
                y_batch = y_batch.to(device)
                y_pred = clf(x_batch_g, x_batch_c).detach()
                avg_val_loss += loss_fn(y_pred, y_batch).item() / len(valid_loader)
                
            elapsed_time = time.time() - start_time 
            scheduler.step(avg_val_loss)
                    
            if avg_val_loss < best_val_loss:
                stop_counts = 0
                best_val_loss = avg_val_loss
                print('Best model: Epoch {} \t loss={:.6f} \t val_loss={:.6f} \t time={:.2f}s'.format(
                    epoch + 1, avg_loss, avg_val_loss, elapsed_time))
                torch.save(clf.state_dict(), 'best-model-parameters.pt')
                
        pred_model = myLSTM(last_num)
        pred_model.load_state_dict(torch.load('best-model-parameters.pt'))
        pred_model.eval()
        
        # validation check ----------------
        oof_epoch = np.zeros([X_valid2_g.size(0), y_train.shape[1]])
        target_epoch = np.zeros([X_valid2_g.size(0), y_train.shape[1]])
        for i, (x_batch_g, x_batch_c, y_batch) in enumerate(valid_loader): 
                y_pred = pred_model(x_batch_g, x_batch_c).sigmoid().detach()
                oof_epoch[i * batch_size:(i+1) * batch_size,:] = y_pred.cpu().numpy()
                target_epoch[i * batch_size:(i+1) * batch_size,:] = y_batch.cpu().numpy()
        print("Fold {} log loss: {}".format(fold+1, mean_log_loss(target_epoch, oof_epoch)))
        scores.append(mean_log_loss(target_epoch, oof_epoch))
        oof[valid_index,:] = oof_epoch
        oof_targets[valid_index,:] = target_epoch
        #-----------------------------------
        
        # test predcition --------------
        test_preds = np.zeros([test_len, y_train.shape[1]])
        for i, (x_batch_g, x_batch_c, ) in enumerate(test_loader): 
            y_pred = pred_model(x_batch_g, x_batch_c).sigmoid().detach()
            test_preds[i * batch_size:(i+1) * batch_size, :] = y_pred.cpu().numpy()
        pred_value += test_preds / n_folds
        # ------------------------------
        
    print("Seed {}".format(seed_))
    for i, ele in enumerate(scores):
        print("Fold {} log loss: {}".format(i+1, scores[i]))
    print("Total log loss: {}".format(mean_log_loss(oof_targets, oof)))

    return oof, pred_value

In [22]:
#lstm1_oof, lstm1_test = modelling_lstm(lstm_train, fn_targets, lstm_test, 0, fn_targets.shape[1])

In [23]:
#check_lstm = np.zeros([targets.shape[0], targets.shape[1]-1])
#check_lstm[cons_train_index,:] = lstm1_oof
#print('OOF log loss: ', log_loss(np.ravel(y), np.ravel(check_lstm)))

# 1st SVM

In [24]:
N_STARTS = 1
N_SPLITS = 5

svm0_oof = np.zeros([len(fn_train), fn_targets.shape[1]])
svm0_test = np.zeros([len(fn_test), fn_targets.shape[1]])

svm1_test = np.zeros([len(fn_test),fn_targets.shape[1]])
svm1_oof = np.zeros([fn_targets.shape[0],fn_targets.shape[1]]) 

for ind in tqdm(range(fn_targets.shape[1])):
    ind_target_sum = fn_targets[:, ind].sum()
    if ind_target_sum >= N_SPLITS:
        for seed in range(N_STARTS):
            skf = StratifiedKFold(n_splits = N_SPLITS, random_state = seed, shuffle = True)

            for n, (train_index, val_index) in enumerate(skf.split(fn_train, fn_targets[:,ind])):
                
                x_tr, x_val = fn_train[train_index], fn_train[val_index]
                y_tr, y_val = fn_targets[train_index,ind], fn_targets[val_index,ind]

                model = SVC(C = 40, cache_size = 2000)
                model.fit(x_tr, y_tr)
                svm0_test[:, ind] += model.decision_function(fn_test) / (N_SPLITS * N_STARTS)
                svm0_oof[val_index, ind] += model.decision_function(x_val) / N_STARTS

        for seed in range(N_STARTS):
            skf = StratifiedKFold(n_splits = N_SPLITS, random_state = seed, shuffle = True)
            
            for n, (train_index, val_index) in enumerate(skf.split(svm0_oof, fn_targets[:,ind])):

                x_tr, x_val = svm0_oof[train_index, ind].reshape(-1, 1), svm0_oof[val_index, ind].reshape(-1, 1)
                y_tr, y_val = fn_targets[train_index,ind], fn_targets[val_index,ind]

                model = LogisticRegression(C = 35, max_iter = 1000)
                model.fit(x_tr, y_tr)
                svm1_test[:, ind] += model.predict_proba(svm0_test[:, ind].reshape(-1, 1))[:, 1] / (N_SPLITS * N_STARTS)
                svm1_oof[val_index, ind] += model.predict_proba(x_val)[:, 1] / N_STARTS

KeyError: "None of [Int64Index([    0,     2,     3,     4,     5,     6,     9,    10,    11,\n               12,\n            ...\n            21937, 21938, 21939, 21940, 21941, 21942, 21943, 21944, 21945,\n            21946],\n           dtype='int64', length=17558)] are in the [columns]"

In [25]:
check_svm = np.zeros([targets.shape[0], targets.shape[1]-1])
check_svm[cons_train_index,:] = svm1_oof
print('OOF log loss: ', log_loss(np.ravel(y), np.ravel(check_svm)))

OOF log loss:  0.11859124020134244


# submission

In [26]:
check = 0.1 * check_svm + 0.2 * check_xgb + 0.2 * check_tabnet + 0.5 * check_mlp
print('OOF log loss: ', log_loss(np.ravel(y), np.ravel(check)))

NameError: name 'check_tabnet' is not defined

In [27]:
sub = pd.read_csv(DATA_DIR + 'sample_submission.csv')
sub.loc[cons_test_index,target_feats] =  0.1 * svm1_test + 0.2 * xgb1_test  + 0.2 * tabnet1_test + 0.5 * pytorch1_test
sub.loc[noncons_test_index,target_feats] = 0
sub.to_csv('submission.csv', index=False)

NameError: name 'xgb1_test' is not defined

# 2nd layer

In [28]:
#train_feat = check[cons_train_index,:].copy()
#test_feat = sub[sub.index.isin(cons_test_index)].copy()
#final = pd.read_csv(DATA_DIR+"sample_submission.csv")
#test_feat.drop("sig_id", axis=1, inplace=True)

In [29]:
#np.random.seed(42)
#xgb_params = {'tree_method': 'gpu_hist', 'max_depth': 5, 'alpha': 0.1,
#          'gamma': 0.3, 'scale_pos_weight': 1, 'learning_rate': 0.01, 
#           'objective':'binary:logistic', 'eval_metric': 'logloss'}

def modelling_xgb(X_train, target_train, X_test, ind):
    
    y_train = target_train[:,ind]
    pred_value = np.zeros(X_test.shape[0])
    valid = np.zeros([X_train.shape[0]])
    X_test = xgb.DMatrix(X_test.values)
    
    n_folds=5
    if y_train.sum() >= n_folds:
        skf=StratifiedKFold(n_splits = n_folds, shuffle=True, random_state=0)
    else:
        skf=KFold(n_splits = n_folds, shuffle=True, random_state=0) 
        
    for i , (train_index, test_index) in enumerate(skf.split(X_train, y_train)):
        X_train2 = X_train[train_index,:]
        y_train2 = y_train[train_index]

        X_test2 = X_train[test_index,:]
        y_test2 = y_train[test_index]
        
        xgb_train = xgb.DMatrix(X_train2, label = y_train2)
        xgb_eval = xgb.DMatrix(X_test2, label = y_test2)
        watchlist = [(xgb_train, "train"), (xgb_eval, "eval")]
        
        clf = xgb.train(
        xgb_params, xgb_train, 100000, watchlist,
        early_stopping_rounds=25, verbose_eval = False
    )

        valid[test_index] = clf.predict(xgb_eval, ntree_limit=clf.best_ntree_limit)    
        pred_value += clf.predict(X_test, ntree_limit=clf.best_ntree_limit) / n_folds
        
    return valid, pred_value

In [30]:
#for ind, target in enumerate(target_feats):
#    valid, pred_value = modelling_xgb(train_feat, fn_targets, test_feat, ind)
#    checkscore2.loc[cons_train_index, target] = valid
#    checkscore2.loc[~checkscore2.index.isin(cons_train_index), target] = 0
#    print("ind: {}, target: {}, all log_loss= {}".format(ind, target, log_loss(targets[target], checkscore2[target])))
#    final.loc[cons_test_index, target] = pred_value
#    final.loc[~final.index.isin(cons_test_index), target] = 0

In [31]:
#scores = []
#for target_col in target_feats:
#    scores.append(log_loss(targets[target_col], checkscore2[target_col]))
#print(np.mean(scores))

In [32]:
#final.to_csv("submission.csv", index=False)